In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import external functions from the functions folder
import sys
sys.path.append('../../functions')
import functions as f

# Import pytorch libraries for the MLP
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
# Set local path to the folder containing the .wav audio files
path = 'C:/Users/lucvo/VScode/Machine_learning/Audio_data/nsynth-valid.jsonwav/nsynth-valid/audio/'

In [4]:
# Generate the dataset
X, y = f.generate_mixed_spectrograms(5000, 3, path=path)

# Flatten the dataset
X = X.reshape(X.shape[0], -1)

# Split into training, validation and test (80/10/10)
X_train, X_val, X_test, y_train, y_val, y_test = f.split_data(X, y, 0.1, 0.1)

100%|██████████| 5000/5000 [02:14<00:00, 37.25it/s] 
